In [1]:
!pip install pandas gspread oauth2client


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import json
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

JSON Fields:

- Solution Text: This contains the actual solution submitted by the student. For programming tasks, it could be the source code.
- Email: The email address of the student who submitted the solution.
- Submission Datetime: The timestamp when the submission was made.
- Course ID: The ID of the course to which the task belongs. This allows us to separate submissions by course.
- User ID: A unique identifier for the user making the submission.
- Visibility: This indicates the visibility status of the submission (e.g., "UNPUBLISHED").
- Task Name: The name of the task being solved (e.g., "Create an Empty Array").
- Status: The result of the submission, indicating whether the submission is correct or wrong.
- ID: A unique identifier for the submission itself.
- Task Type: The type of task (e.g., "edu", "theory"). This can help distinguish between different types of tasks.
- Task ID: A unique identifier for the task being solved.

In [3]:
# Task 1: Parse the file and load data into a pandas DataFrame
def load_submission_data(file_path):
    # Load the JSON file
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    # Create a DataFrame from the list of submissions
    df = pd.DataFrame(data)
    return df


In [15]:
# Task 2: Update Google Spreadsheet by Course ID in separate tabs, add new lines and columns if necessary
def update_google_sheet_incremental(df, credentials_file, sheet_name):
    # Authenticate Google Sheets API using the service account credentials
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_file, scope)
    client = gspread.authorize(creds)
    
    # Open the Google Spreadsheet by name
    spreadsheet = client.open(sheet_name)
    
    # Get unique Course IDs to separate submissions
    course_ids = df['Course ID'].unique()

    # Filter non 'eud' submissions
    df = df[df['Task Type'] == 'edu']

    for course_id in course_ids:
        # Filter submissions for the current Course ID
        course_df = df[df['Course ID'] == course_id]
        
        # Get unique emails (students) and task names (tasks)
        emails = course_df['Email'].unique()
        task_names = course_df['Task Name'].unique()
        
        # Sheet title for the current course
        sheet_title = f"Course {course_id}"
        
        # Check if the worksheet for this course already exists
        try:
            worksheet = spreadsheet.worksheet(sheet_title)
        except gspread.exceptions.WorksheetNotFound:
            # Create a new worksheet for the course if it doesn't exist
            worksheet = spreadsheet.add_worksheet(title=sheet_title, rows=1000, cols=100)
            worksheet.append_row(['Email'] + list(task_names))  # Add headers if a new worksheet is created
        
        # Get existing data from the sheet (including headers)
        existing_data = worksheet.get_all_values()
        if existing_data:
            # Extract headers (task names) and student emails
            existing_emails = [row[0] for row in existing_data[1:]]  # First column, skip header
            existing_headers = existing_data[0]  # First row
        else:
            existing_emails = []
            existing_headers = ['Email']
        
        # Add new columns (task names) if not already present
        for task_name in task_names:
            if task_name not in existing_headers:
                worksheet.update_cell(1, len(existing_headers) + 1, task_name)  # Add to header
                existing_headers.append(task_name)
        
        # Add or update rows for each student submission
        for email in emails:
            if email in existing_emails:
                # Student already exists, update their task results
                row_idx = existing_emails.index(email) + 2  # Row index (offset by header)
            else:
                # New student, append a new row
                row_idx = len(existing_emails) + 2
                worksheet.append_row([email] + [0] * (len(existing_headers) - 1))
                existing_emails.append(email)

            # Update the results for each task
            for _, row in course_df[course_df['Email'] == email].iterrows():
                if row['Status'] == 'correct':
                    col_idx = existing_headers.index(row['Task Name']) + 1  # Column index
                    worksheet.update_cell(row_idx, col_idx, 1)  # Update to 1 for correct status
    
    print("Data incrementally updated successfully for all courses.")


In [17]:
# Main script execution
submission_file = './data/query_result_2024-09-11T13_34_44.896091Z.json'
credentials_file = './data/aiclub-435409-2881eaca101a.json'
sheet_name = 'AI club gradebook'
    
# Load the submissions into a DataFrame
df = load_submission_data(submission_file)
    
# Update Google Sheet with the processed data
update_google_sheet_incremental(df, credentials_file, sheet_name)

Data incrementally updated successfully for all courses.
